In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Task 1: Loading and Reading the data

In [ ]:
path = '/kaggle/input/d/db333333/sales-analysis-day1/'
files = os.listdir(path)  # listing the files in the directory

In [ ]:
# Join all the monthly sales dataframes
sales_dfs = [pd.read_csv(path + file) for file in files]
df = pd.concat(sales_dfs)

In [ ]:
df.head(10)  # Print the first 10 rows

In [ ]:
# Get brief info about the joined dataframe
df.info()

In [ ]:
df.replace("Price Each", np.NaN, inplace=True) # Replace all "Price Each" with NaN
df.isnull().sum()  # displaying number of missing values in the dataframe 

In [ ]:
df.dropna(inplace=True)  # Drop the missing values
df.isnull().sum()

In [ ]:
# Converting data type of price, quantity and order date columns
df['Price Each'] = df['Price Each'].astype('float32')
df['Quantity Ordered'] = df['Quantity Ordered'].astype('int32')
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M')  # changing the order date to datetime and specifying its format to parse it

df.info()  # displaying decsription of columns

# Task 2: Get the month with highest sales

In [ ]:
import calendar

monthly_total_sales = {}

for month in range(1, 13):
    monthly_data = df[df['Order Date'].dt.month == month]  # filtering data based on month
    sales = (monthly_data['Price Each'] * monthly_data['Quantity Ordered']).sum() # price * quantity of each row in the filtered data
    monthly_total_sales.update( { calendar.month_name[month]: round(sales, 2) } )  # inserting the monthly sales in the dict
    
monthly_total_sales

In [ ]:
# Plotting the monhtly sales as bar graph
plt.figure(figsize=(13, 5))
plt.title("Total Sales by month")
plt.xlabel("Months")
plt.ylabel("Total Sales in millions")
plt.bar(x=monthly_total_sales.keys(), height=monthly_total_sales.values())
plt.plot()

# Task 3: Which city had highest sales

In [ ]:
areas = df['Purchase Address'].apply( lambda x: x[: x.index(',')] ) 
cities = df['Purchase Address'].apply( lambda x: x[x.index(',') + 2 : x.rindex(',')] ) 
states = df['Purchase Address'].apply( lambda x: x[x.rindex(',') + 2 : -6] ) 
pincodes = df['Purchase Address'].apply( lambda x: x[-6:] ) 

# Creating new columns areas, cities, states and pincodes in the dataframe
df['Area'] = areas
df['City'] = cities
df['State'] = states
df['Pincode'] = pincodes

# Dropping the column Purchase address
df.drop('Purchase Address', axis=1, inplace=True)

df.head(10)

In [ ]:
df.info()

In [ ]:
# Printing all the states
all_states = df['State'].unique()
print("States:", all_states)

In [ ]:
# Printing all the cities
all_cities = df['City'].unique()
print("Cities:", all_cities)

In [ ]:
city_sales = {}

for city in all_cities:
    data = df[df['City'] == city]  # Filter the data city-wise
    sales = (data['Price Each'] * data['Quantity Ordered']).sum()  # Price * Quantity
    city_sales.update({ city: round(sales, 2) }) # Add the sales to the dictionary

city_sales

In [ ]:
# Plotting the city wise sales as bar graph
plt.figure(figsize=(13, 5))
plt.title("Total sales city wise")
plt.xlabel("American cities")
plt.ylabel("Total sales in millions")
plt.bar(x=city_sales.keys(), height=city_sales.values())
plt.plot()

# Task 4: Which products are often sold together? Get top 3 products that are sold together

In [ ]:
# Getting all the products grouped together with same product ID
df_orderId_grouped = df.groupby('Order ID').agg({ 
    'Product': ','.join, 
    'Quantity Ordered': 'sum' 
})

# Getting the items grouped together with same product combos and displaying their total quantity 
combos_grouped_quantity_data = df_orderId_grouped.groupby('Product', as_index=False).agg({
    'Quantity Ordered': 'sum'
})

combos_grouped_quantity_data

In [ ]:
item_combos = []

for entry in combos_grouped_quantity_data.values:
    if len(entry[0].split(',')) == 2:
        item_combos.append([entry[0], entry[1]])

        
sorted(item_combos, key=lambda x: x[1], reverse=True)[:3]  # displaying the top 3 item combos

# Task 5: Which Product is most sold and which one is least sold?

In [ ]:
# Displaying all the products in the data
products = df['Product'].unique()
products

In [ ]:
product_quantities = {}

# Adding the units sold by iterating through orders
for product in products:
    data = df[df['Product'] == product]  # filtering data based on product
    units_sold = data['Quantity Ordered'].sum() # getting all the units sold for each product
    product_quantities.update({ product: units_sold })  # inserting the product units sold

product_quantities

In [ ]:
# PLotting the product units sold
plt.figure(figsize=(12,8))
plt.title("Number of Product units sold")
plt.xlabel("Products")
plt.ylabel("Units sold")
plt.bar(x=product_quantities.keys(), height=product_quantities.values())
plt.xticks(rotation=45, fontsize=8)
plt.tight_layout()
plt.plot()

# Task 6: What time ads should be displayed?

In [ ]:
hour_orders = {}

# Checking the orders in 24 hours
for hour in range(24):
    hour_orders.update({ hour: len(df[df['Order Date'].dt.hour == hour]) })

In [ ]:
# Plotting the hourly sales as bar graph
plt.figure(figsize=(13, 5))
plt.title("Hourly traffic")
plt.xlabel("Hours")
plt.ylabel("Number of Orders")
plt.bar(x=hour_orders.keys(), height=hour_orders.values())
plt.xticks(list(range(0, 24)))
plt.plot()